In [1]:
pip show azureml-train-automl

Name: azureml-train-automl
Version: 1.48.0
Summary: Used for automatically finding the best machine learning model and its parameters.
Home-page: https://docs.microsoft.com/python/api/overview/azure/ml/?view=azure-ml-py
Author: Microsoft Corp
Author-email: None
License: https://aka.ms/azureml-sdk-license
Location: /anaconda/envs/azureml_py38/lib/python3.8/site-packages
Requires: azureml-automl-runtime, azureml-train-automl-client, azureml-train-automl-runtime, azureml-dataset-runtime, azureml-automl-core
Required-by: azureml-automl-dnn-nlp
Note: you may need to restart the kernel to use updated packages.


In [2]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

Ready to use Azure ML 1.48.0 to work with project


In [3]:
from azureml.core import Dataset
from azureml.data.datapath import DataPath

default_ds = ws.get_default_datastore()

if 'churn dataset' not in ws.datasets:
    Dataset.File.upload_directory(src_dir='data',
                              target=DataPath(default_ds, 'churn-data/')
                              )

    #Create a tabular dataset from the path on the datastore (this may take a short while)
    tab_data_set = Dataset.Tabular.from_delimited_files(path=(default_ds, 'churn-data/*.csv'))

    # Register the tabular dataset
    try:
        tab_data_set = tab_data_set.register(workspace=ws, 
                                name='telco dataset',
                                description='chrun data',
                                tags = {'format':'CSV'},
                                create_new_version=True)
        print('Dataset registered.')
    except Exception as ex:
        print(ex)
else:
    print('Dataset already registered.')

# Split the dataset into training and validation subsets
churn_ds = ws.datasets.get("telco dataset")
train_ds, test_ds = churn_ds.random_split(percentage=0.7, seed=123)
print("Data ready!")

Validating arguments.
Arguments validated.
Uploading file to churn-data/
Uploading an estimated of 3 files
Target already exists. Skipping upload for churn-data/.amlignore
Target already exists. Skipping upload for churn-data/.amlignore.amltmp
Target already exists. Skipping upload for churn-data/telco-csv.csv
Uploaded 0 files
Creating new dataset
Dataset registered.
Data ready!


In [4]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "your-churn-compute-cluster"

try:
    # Check for existing compute target
    pipeline_cluster = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # If it doesn't already exist, create it
    try:
        compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2', max_nodes=2)
        pipeline_cluster = ComputeTarget.create(ws, cluster_name, compute_config)
        pipeline_cluster.wait_for_completion(show_output=True)
    except Exception as ex:
        print(ex)

Found existing cluster, use it.


In [5]:
import azureml.train.automl.utilities as automl_utils

for metric in automl_utils.get_primary_metrics('classification'):
    print(metric)

precision_score_weighted
AUC_weighted
average_precision_score_weighted
accuracy
norm_macro_recall


In [6]:
from azureml.train.automl import AutoMLConfig

automl_config = AutoMLConfig(name='Automated ML Experiment',
                             task='classification',
                             compute_target=pipeline_cluster,
                             training_data = train_ds,
                             validation_data = test_ds,
                             label_column_name='churn',
                             iterations=4,
                             primary_metric = 'AUC_weighted',
                             max_concurrent_iterations=2,
                             featurization='auto'
                             )

print("Ready for Auto ML run.")

Ready for Auto ML run.


In [7]:
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

print('Submitting Auto ML experiment...')
automl_experiment = Experiment(ws, 'mslearn-churn-automl-sdk')
automl_run = automl_experiment.submit(automl_config)
RunDetails(automl_run).show()
automl_run.wait_for_completion(show_output=True)

Submitting Auto ML experiment...
Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
mslearn-churn-automl-sdk,AutoML_0b639f79-7864-4c18-863a-c91571486e74,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
mslearn-churn-automl-sdk,AutoML_0b639f79-7864-4c18-863a-c91571486e74,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetFeaturizationCompleted. Completed fit featurizers and featurizing the dataset.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       DONE
DESCRIPTION:  If the missing values are expected, let the run complete. Otherwise cancel the current run and use a script to customize the handling of mi

{'runId': 'AutoML_0b639f79-7864-4c18-863a-c91571486e74',
 'target': 'your-churn-compute-cluster',
 'status': 'Completed',
 'startTimeUtc': '2023-02-03T11:50:36.070583Z',
 'endTimeUtc': '2023-02-03T11:55:52.42427Z',
 'services': {},
 'properties': {'num_iterations': '4',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'AUC_weighted',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': None,
  'target': 'your-churn-compute-cluster',
  'AMLSettingsJsonString': '{"path":null,"name":"Automated ML Experiment","subscription_id":"bd5c51d3-de0b-41fc-99f7-375e6737efda","resource_group":"project_dp100","workspace_name":"project","region":"eastus","compute_target":"your-churn-compute-cluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":true,"enable_parallel_run":false,"num_procs":null,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":

In [8]:
for run in automl_run.get_children():
    print('Run ID', run.id)
    for metric in run.get_metrics():
        print('\t', run.get_metrics(metric))

Run ID AutoML_0b639f79-7864-4c18-863a-c91571486e74_3
	 {'matthews_correlation': 0.3268857062272351}
	 {'f1_score_weighted': 0.7465789665954499}
	 {'recall_score_weighted': 0.7740863787375415}
	 {'weighted_accuracy': 0.8659989608155806}
	 {'AUC_weighted': 0.7710129929860873}
	 {'precision_score_weighted': 0.7523803159328419}
	 {'average_precision_score_weighted': 0.8008426427906363}
	 {'balanced_accuracy': 0.6266241232608946}
	 {'f1_score_micro': 0.7740863787375415}
	 {'average_precision_score_macro': 0.7143152310633113}
	 {'AUC_macro': 0.7710129929860872}
	 {'average_precision_score_micro': 0.8429124206935179}
	 {'accuracy': 0.7740863787375415}
	 {'precision_score_macro': 0.7109674329501916}
	 {'precision_score_micro': 0.7740863787375415}
	 {'log_loss': 0.4972722347262396}
	 {'AUC_micro': 0.8502224037262283}
	 {'norm_macro_recall': 0.2532482465217891}
	 {'recall_score_macro': 0.6266241232608946}
	 {'f1_score_macro': 0.6416164728953635}
	 {'recall_score_micro': 0.7740863787375415}
	 {'c

In [10]:
best_run, fitted_model = automl_run.get_output()
print(best_run)
print('\nBest Model Definition:')
print(fitted_model)
print('\nBest Run Transformations:')
for step in fitted_model.named_steps:
    print(step)
print('\nBest Run Metrics:')
best_run_metrics = best_run.get_metrics()
for metric_name in best_run_metrics:
    metric = best_run_metrics[metric_name]
    print(metric_name, metric)

PermissionError: [Errno 1] Operation not permitted: '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/nimbusml/internal/libs/pybridge.so'